### IMPORTS

In [1]:
import json
import subprocess
import sys
import time
import hashlib
import base64

In [2]:
from web3 import Web3

ModuleNotFoundError: No module named 'cytoolz'

In [3]:
import ipynb.fs.full.oracle as oracle

### FIX FOR WINDOWS ENVIRONMENT

In [ ]:
#import pkg_resources.py2_warn

### LOGGING BASELINE

In [ ]:
def log(msg):
    print('[LAUNCHER] ' + msg)

### LOAD DATA FROM EXTERNAL FILES

In [ ]:
with open('config/settings.json') as json_file:
    settings = json.load(json_file)

In [ ]:
with open('config/identifier.json') as json_file:
    device_info = json.load(json_file)

In [ ]:
with open('config/latest.json') as json_file:
    latest = json.load(json_file)

In [ ]:
log('READ DATA FROM EXTERNAL FILES')

### CONNECT TO BLOCKCHAIN VIA WEBSOCKET

In [ ]:
web3 = Web3(Web3.WebsocketProvider('ws://' + settings['gateway']['host'] + ':' + settings['gateway']['port']))

In [ ]:
if web3.isConnected():
    log('GATEWAY CONNECTION ESTABLISHED')
else:
    print('COULD NOT CONNECT WITH GATEWAY. ABORTING...')
    exit

### EXTRACT THE WHISPER API

In [ ]:
shh = web3.geth.shh

### GENERATE A FRESH WHISPER ID

In [ ]:
whisper_id = shh.newKeyPair()

In [ ]:
log('WHISPER ID GENERATED')

### HASH DEVICE IDENTIFIER

In [ ]:
def hash_id(data):
    
    # REMOVE WHITESPACES
    to_string = json.dumps(data, sort_keys=False, indent=2)
    
    # ENCODE THE STRING WITH UTF8
    encoded = to_string.encode('utf-8')
    
    # HASH ENCODED DATA
    hashed = hashlib.sha256(encoded).hexdigest()
    
    return hashed

### CREATE USABLE DEVICE OUTLINE

In [ ]:
class create_device():
    def __init__(self, data):
        self.hash = hash_id(data)
    
    # LOCATE & SET DEVICE CONTRACT ADDRESS
    def set_contract(self, contract):
        self.contract = contract
    
    # REDIRECT TO PARENT
    def read(self, details):
        return self.contract.read(details)
    
    # REDIRECT TO PARENT
    def write(self, details):
        return self.contract.write(details)
    
    # REDIRECT TO PARENT
    def event(self, name):
        return self.contract.event(name)

In [ ]:
device = create_device(device_info)

In [ ]:
log('DEVICE ID HASHED')

### UNIFORM SMART CONTRACT OUTLINE

In [ ]:
class contract:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = web3.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, details):
        
        # WITH PARAMS
        if ('params' in details):
            return self.contract.functions[details['func']](details['params']).call()
        
        # WITHOUT PARAMS
        else:
            return self.contract.functions[details]().call()
    
    # WRITE TO CONTRACT
    def write(self, details):
        try:
            
            # CREATE BASE TRANSACTION
            tx = {
                'from': settings['keys']['public'],
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                )
            }
            
            # ESTIMATE GAS VALUE & STITCH IN REMAINING PROPS
            tx['gas'] = web3.eth.estimateGas(tx)
            tx['gasPrice'] = web3.toWei(20, 'gwei')
            tx['nonce'] = web3.eth.getTransactionCount(settings['keys']['public'])

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = web3.eth.account.sign_transaction(tx,
                private_key = settings['keys']['private']
            )

            # SEND THE TRANSACTION
            tx_hash = web3.eth.sendRawTransaction(signed.rawTransaction)

            # WAIT FOR IT TO BE MINED
            return web3.eth.waitForTransactionReceipt(tx_hash, 500)
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error
    
    # EVENT FILTER
    def event(self, name):
        return self.contract.events[name].createFilter(fromBlock="latest")

### SERIALIZE NECESSARY MANAGER CONTRACTS

In [ ]:
device_manager = contract(latest['devicemanager'])

In [ ]:
task_manager = contract(latest['taskmanager'])

In [ ]:
log('MANAGER CONTRACTS SERIALIZED')

### SERIALIZE DEVICE CONTRACT

In [ ]:
temp_contract = contract({
    'address': device_manager.read({
        'func': 'fetch_device',
        'params': device.hash
    }),
    'abi': latest['device']['abi']
})

In [ ]:
device.set_contract(temp_contract)

### MAKE SURE THE DEVICE IS REGISTERED

In [ ]:
if (temp_contract.address != '0x0000000000000000000000000000000000000000'):
    log('DEVICE CONTRACT SERIALIZED')
else:
    log('THE DEVICE ISNT REGISTERED. ABORTING...')

### EVENT HELPER FUNCS

In [ ]:
def filter_backlog(data):
    
    # FILTER ZEROS
    filtered = filter(lambda x: x != '0x0000000000000000000000000000000000000000', data)
    
    # CONVERT TO LIST & RETURN
    return list(filtered)

In [ ]:
def encode(data):
    
    # STRINGIFY & CONVERT TO BYTES
    stringified = json.dumps(data)
    to_bytes = str.encode(stringified)
    
    # ENCODE
    encoded = base64.b64encode(to_bytes)
    
    # RETURN AS STRING
    return encoded.decode()

In [ ]:
def decode(compressed):
    
    # ATTEMPT TO DECODE & PARSE AS JSON
    try:
        to_bytes = base64.b64decode(compressed)
        return json.loads(to_bytes)
    
    # OTHERWISE, RETURN EMPTY OBJECT
    except:
        return {}

In [ ]:
def compare_services(data, base):
    
    # RESULT CONTAINER
    result = []
    
    # CONVERT BASE TO SET FOR QUICKER LOOKUPS
    base = set(base)
    
    # LOOP THROUGH & APPEND CHECK RESULT
    for key in data:
        result.append(key in base)
    
    # FINALLY RETURN
    return result

In [ ]:
def compare_discovery(data, base):
    
    # RESULT CONTAINER
    result = []
    
    # LOOP THROUGH DATA KEYS
    for key in data:
        
        # IF THE KEY EXISTS IN THE BASE DICT
        if key in base:
            
            # IF THE VALUE IS SAME IN BOTH DATASET
            if data[key] == base[key]:
                result.append(True)
                
            # OTHERWISE, DEFAULT TO FALSE
            else:
                result.append(False)
                
        # OTHERWISE, DEFAULT TO FALSE
        else:
            result.append(False)
            
    # FINALLY RETURN RESULT
    return result

### GLOBAL TASK BACKLOG

In [ ]:
try:
    raw = device.read('details')[1]
except:
    log('THE DEVICE HASH IS NOT REGISTERED')

In [ ]:
backlog = filter_backlog(raw)

In [ ]:
log('TASKS IN BACKLOG: ' + str(len(backlog)))

### GLOBAL ACTIVE STATUS 

In [ ]:
active = device.read('active')

In [ ]:
log('CURRENT ACTIVE STATUS: ' + str(active))

### GLOBAL DISCOVERY STATUS

In [ ]:
discoverable = device.read('discoverable')

In [ ]:
log('CURRENT DISCOVERABLE STATUS: ' + str(discoverable))

### GLOBAL DISCOVERY PARAMS

In [ ]:
compressed = device.read('tags')

In [ ]:
discovery_config = decode(compressed)

In [ ]:
log('DISCOVERY PARAMS FETCHED')

### GLOBAL SERVICES

In [ ]:
services = device.read('fetch_services')

In [ ]:
log('SERVICES FETCHED')

### EVENT FUNCTIONS

In [ ]:
def update_details(event):
    
    # FETCH GLOBAL VARS
    global active
    global discoverable
    global backlog
    global discovery_config
    global services
    
    # EXTRACT RELEVANT VALUES
    latest_active = event['args']['active']
    latest_discoverable = event['args']['discoverable']
    latest_config = decode(event['args']['tags'])
    latest_services = event['args']['services']

    # IF ACTIVE STATUS HAS CHANGED
    if (latest_active != active):
        
        # UPDATE ACTIVE STATUS
        active = latest_active
        
        # SEND MSG
        log('ACTIVE STATUS CHANGED TO: ' + str(latest_active))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_discoverable != discoverable):
        
        # UPDATE ACTIVE STATUS
        discoverable = latest_discoverable
        
        # SEND MSG
        log('DISCOVERABLE STATUS CHANGED TO: ' + str(latest_discoverable))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_config != discovery_config):
        
        # UPDATE ACTIVE STATUS
        discovery_config = latest_config
        
        # SEND MSG
        log('DISCOVERY CONFIG CHANGED')
        
    # IF SERVICES HAVE CHANGED
    if (latest_services != services):
        
        # UPDATE ACTIVE STATUS
        services = latest_services
        
        # SEND MSG
        log('SERVICES HAVE CHANGED')
        
    # UPDATE BACKLOG
    raw_backlog = event['args']['backlog']
    backlog = filter_backlog(raw_backlog)

In [ ]:
def update_middleware():
    
    # PRINT REACTION
    log('MIDDLEWARE UPDATE TRIGGERED')

    # TRIGGER UPDATE SCRIPT
    # subprocess.call('./patcher')

    # CLOSE LANCHER
    # sys.exit(0)

In [ ]:
def perform_task(task):
    
    # SHOW MSG
    log('STARTING TASK: ' + task)

    # TASK RETURN PARAMS
    ipfs = 'QmWATWQ7fVPP2EFGu71UkfnqhYXDYH566qy47CnJDgvs8u'
    key = '0x4f7a87EE7A53ae8606e80FE96a47038DF8ab7956'
    
    # ADD WHATEVER TASK MODULE HERE

    # SUBMIT THE TASK RESULT
    task_manager.write({
        'func': 'complete',
        'params': [task, ipfs, key]
    })

    # SHOW MSG
    log('TASK COMPLETED')

In [ ]:
def process_message(event):

    # SERIALIZE EVENT PARAMS
    author = blockchain.toHex(event['sig'])
    payload = blockchain.toText(event['payload'])

    # DECODE THE PAYLOAD
    data = decode(payload)
    
    # REQUIRED KEYS FOR VALID MESSAGE
    required = ['type', 'services', 'discovery']

    # DECODED KEYS
    keys = list(data.keys())
    
    # THE REQUEST KEYWORD FOR THE PAYLOAD TYPE
    keyword = 'request'
    
    # IF THE KEYSETS MATCH & THE TYPE IS A REQUEST
    if (required == keys and data['type'] == keyword):
        
        # CHECK MATCHES IN SERVICES & DISCOVERY PARAMS
        services_result = compare_services(data['services'], services)
        discovery_result = compare_discovery(data['discovery'], discovery_config)
        
        # COMBINE LISTS FOR VERIFICATION
        result = services_result + discovery_result
        
        # IF EVERYTHING MATCHED
        if (result.count(False) == 0):
            
            # SHOW MSG
            log('DISCOVERY REQUEST DETECTED')
            
            # ENCODE A JSON RESPONSE
            response = encode({
                'type': 'response',
                'source': payload,
                'device': device.hash
            })
            
            # SLEEP FOR 2 SECONDS
            time.sleep(2)

            # RESPOND TO REQUEST
            shh.post({
                'symKeyID': settings['whisper']['topic']['key'],
                'payload': blockchain.toHex(text=response),
                'topic': blockchain.toHex(text=settings['whisper']['topic']['name']),
                'sig': whisper_id,
                'powTarget': 2.5,
                'powTime': 2
            })

### START LISTENING FOR EVENTS

In [ ]:
log('AWAITING EVENTS...\n')

### CONTRACT EVENTS

In [ ]:
update_event = device.event('middleware')

In [ ]:
changes_event = device.event('changes')

In [ ]:
assignment_event = device.event('assignment')

In [ ]:
message_event = shh.newMessageFilter({
    'topic': web3.toHex(text=settings['whisper']['topic']['name']),
    'symKeyID': settings['whisper']['topic']['key']
})

### EVENT LOOP

In [ ]:
try:
    while(True):
    
        # FETCH THE GLOBAL PARAMS
        global backlog
        global active
        global discoverable

        # ACTIVE STATUS EVENT
        for event in changes_event.get_new_entries():
            update_details(event)

        # UPDATE MIDDLEWARE EVENT
        for event in update_event.get_new_entries():
            update_middleware()

        # IF THE DEVICE IS SET TO ACTIVE
        if (active):

            # PERFORM TASKS IN BACKLOG
            for task in backlog:
                perform_task(task)
        
        # IF THE DEVICE IS DISCOVERABLE
        if (discoverable):
            
            # TRACK WHISPER REQUESTS
            for event in shh.getMessages(message_event):
                process_message(event)

# WHEN THE PROCESS IS KILLED...
except KeyboardInterrupt:
    print('\nThe process was manually stopped...')
    pass